In [1]:
import platform,os
os_name = platform.system()
hdfs_home = "/user/" + os.getenv("HOME").split("/")[2]

In [2]:
print(hdfs_home)
print(os_name)

/user/kranthidr5051
Linux


In [3]:
path = hdfs_home+"/dataSets/spark-guide/activity-data/"

In [4]:
spark_home = "/usr/hdp/current/spark2-client"

In [5]:
import findspark
findspark.init(spark_home)
#findspark.init()
findspark.find()

'/usr/hdp/current/spark2-client'

In [6]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("sg21-streamingBasics").getOrCreate()
#spark = SparkSession.builder.master("local[*]").appName("sg21-streamingBasics").getOrCreate()

In [7]:
sc = spark.sparkContext

In [8]:
for x in sc._conf.getAll():
    if '/proxy/' in x[1]:
        print(x[1])

http://ip-172-31-35-141.ec2.internal:8088/proxy/application_1534772926501_6717


In [9]:
static = spark.read.json(path).cache()
dataSchema = static.schema

In [10]:
static.rdd.getNumPartitions()

12

In [11]:
static.show(10,False)

+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
|Arrival_Time |Creation_Time      |Device  |Index|Model |User|gt   |x           |y           |z           |
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
|1424686735090|1424686733090638193|nexus4_1|18   |nexus4|g   |stand|3.356934E-4 |-5.645752E-4|-0.018814087|
|1424686735292|1424688581345918092|nexus4_2|66   |nexus4|g   |stand|-0.005722046|0.029083252 |0.005569458 |
|1424686735500|1424686733498505625|nexus4_1|99   |nexus4|g   |stand|0.0078125   |-0.017654419|0.010025024 |
|1424686735691|1424688581745026978|nexus4_2|145  |nexus4|g   |stand|-3.814697E-4|0.0184021   |-0.013656616|
|1424686735890|1424688581945252808|nexus4_2|185  |nexus4|g   |stand|-3.814697E-4|-0.031799316|-0.00831604 |
|1424686736094|1424686734097840342|nexus4_1|218  |nexus4|g   |stand|-7.324219E-4|-0.013381958|0.01109314  |
|1424686736294|1424688582347

In [12]:
static.printSchema()

root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



In [13]:
static.groupby("User").count().show()

+----+------+
|User| count|
+----+------+
|   g|733387|
|   f|736442|
|   e|768182|
|   h|618617|
|   d|649961|
|   c|617237|
|   i|740429|
|   b|729907|
|   a|646829|
+----+------+



In [14]:
static.groupby("Device").count().show()

+--------+-------+
|  Device|  count|
+--------+-------+
|nexus4_1|3091811|
|nexus4_2|3149180|
+--------+-------+



In [15]:
static.groupby("Model").count().show()

+------+-------+
| Model|  count|
+------+-------+
|nexus4|6240991|
+------+-------+



In [16]:
spark.conf.get("spark.sql.streaming.schemaInference")

u'false'

In [17]:
spark.conf.get("spark.sql.shuffle.partitions")

u'200'

In [18]:
spark.conf.set("spark.sql.streaming.schemaInference", "true")
spark.conf.set("spark.sql.shuffle.partitions", "8")

In [19]:
spark.conf.get("spark.sql.shuffle.partitions")

u'8'

In [20]:
spark.conf.get("spark.sql.streaming.schemaInference")

u'true'

In [21]:
# COMMAND ----------
#streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1).json(path)
streaming = spark.readStream.option("maxFilesPerTrigger", 1).json(path)

In [22]:
# COMMAND ----------
activityCounts = streaming.groupBy("gt").count()

In [23]:
# COMMAND ----------
activityQuery = activityCounts.writeStream.queryName("activity_counts")\
  .format("memory").outputMode("complete")\
  .start()

In [24]:
# COMMAND ----------
from time import sleep
for x in range(3):
    spark.sql("SELECT * FROM activity_counts").show()
    sleep(5)

+---+-----+
| gt|count|
+---+-----+
+---+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|      bike|53984|
|      null|52239|
|  stairsup|52260|
|     stand|56924|
|       sit|61547|
|      walk|66280|
|stairsdown|46825|
+----------+-----+

+----------+------+
|        gt| count|
+----------+------+
|      bike|140370|
|      null|135827|
|  stairsup|135888|
|     stand|148005|
|       sit|160006|
|      walk|172327|
|stairsdown|121732|
+----------+------+



In [25]:
static.groupby("gt").count().show()

+----------+-------+
|        gt|  count|
+----------+-------+
|      bike| 863710|
|      null| 835725|
|  stairsup| 836598|
|     stand| 910783|
|       sit| 984714|
|      walk|1060402|
|stairsdown| 749059|
+----------+-------+



In [26]:
# COMMAND ----------
from pyspark.sql.functions import expr
simpleTransform = streaming.withColumn("stairs", expr("gt like '%stairs%'"))\
  .where("stairs")\
  .where("gt is not null")\
  .select("gt", "model", "arrival_time", "creation_time")\
  .writeStream\
  .queryName("simple_transform")\
  .format("memory")\
  .outputMode("append")\
  .start()

In [27]:
for x in range(3):
    spark.sql("SELECT * FROM simple_transform").show(5)
    sleep(5)

+---+-----+------------+-------------+
| gt|model|arrival_time|creation_time|
+---+-----+------------+-------------+
+---+-----+------------+-------------+

+--------+------+-------------+-------------------+
|      gt| model| arrival_time|      creation_time|
+--------+------+-------------+-------------------+
|stairsup|nexus4|1424687983719|1424687981726802718|
|stairsup|nexus4|1424687984000|1424687982009853255|
|stairsup|nexus4|1424687984404|1424687982411977009|
|stairsup|nexus4|1424687984805|1424687982814351277|
|stairsup|nexus4|1424687985210|1424687983217500861|
+--------+------+-------------+-------------------+
only showing top 5 rows

+--------+------+-------------+-------------------+
|      gt| model| arrival_time|      creation_time|
+--------+------+-------------+-------------------+
|stairsup|nexus4|1424687983719|1424687981726802718|
|stairsup|nexus4|1424687984000|1424687982009853255|
|stairsup|nexus4|1424687984404|1424687982411977009|
|stairsup|nexus4|1424687984805|1424687

In [28]:
spark.streams.active

In [29]:
# COMMAND ----------
deviceModelStats = streaming.cube("gt", "model").avg()\
  .drop("avg(Arrival_time)")\
  .drop("avg(Creation_Time)")\
  .drop("avg(Index)")\
  .writeStream.queryName("device_counts").format("memory")\
  .outputMode("complete")\
  .start()

In [30]:
for x in range(3):
    spark.sql("SELECT * FROM device_counts").show(5)
    sleep(5)

+---+-----+------+------+------+
| gt|model|avg(x)|avg(y)|avg(z)|
+---+-----+------+------+------+
+---+-----+------+------+------+

+--------+------+--------------------+--------------------+--------------------+
|      gt| model|              avg(x)|              avg(y)|              avg(z)|
+--------+------+--------------------+--------------------+--------------------+
|    null|nexus4|0.002921030154575719|-0.00836616384160...|-0.00897552622204...|
|    walk|  null|0.001970352086338262| 7.48966684595554E-5|-0.00149828380428...|
|stairsup|  null|-0.02623301318863349|-0.01385931765291...|-0.09395009728019485|
|stairsup|nexus4|-0.02623301318863349|-0.01385931765291...|-0.09395009728019485|
|    bike|nexus4| 0.02351254447093367|-0.01304747996973...|-0.08360475809007034|
+--------+------+--------------------+--------------------+--------------------+
only showing top 5 rows

+--------+------+--------------------+--------------------+--------------------+
|      gt| model|              a

In [31]:
# COMMAND ----------

historicalAgg = static.groupBy("gt", "model").avg()

In [32]:
deviceModelStats = streaming.drop("Arrival_Time", "Creation_Time", "Index")\
  .cube("gt", "model").avg()\
  .join(historicalAgg, ["gt", "model"])\
  .writeStream.queryName("device_counts1").format("memory")\
  .outputMode("complete")\
  .start()

In [33]:
for x in range(3):
    spark.sql("SELECT * FROM device_counts1").show(5)
    sleep(5)

+---+-----+------+------+------+-----------------+------------------+----------+------+------+------+
| gt|model|avg(x)|avg(y)|avg(z)|avg(Arrival_Time)|avg(Creation_Time)|avg(Index)|avg(x)|avg(y)|avg(z)|
+---+-----+------+------+------+-----------------+------------------+----------+------+------+------+
+---+-----+------+------+------+-----------------+------------------+----------+------+------+------+

+---+-----+------+------+------+-----------------+------------------+----------+------+------+------+
| gt|model|avg(x)|avg(y)|avg(z)|avg(Arrival_Time)|avg(Creation_Time)|avg(Index)|avg(x)|avg(y)|avg(z)|
+---+-----+------+------+------+-----------------+------------------+----------+------+------+------+
+---+-----+------+------+------+-----------------+------------------+----------+------+------+------+

+--------+------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+--------------------

In [34]:
spark.streams.active

In [35]:
# # COMMAND ----------

# # Subscribe to 1 topic
# df1 = spark.readStream.format("kafka")\
#   .option("kafka.bootstrap.servers", "host1:port1,host2:port2")\
#   .option("subscribe", "topic1")\
#   .load()
    

In [36]:
# # Subscribe to multiple topics
# df2 = spark.readStream.format("kafka")\
#   .option("kafka.bootstrap.servers", "host1:port1,host2:port2")\
#   .option("subscribe", "topic1,topic2")\
#   .load()
    

In [37]:
# # Subscribe to a pattern
# df3 = spark.readStream.format("kafka")\
#   .option("kafka.bootstrap.servers", "host1:port1,host2:port2")\
#   .option("subscribePattern", "topic.*")\
#   .load()

In [38]:
# # COMMAND ----------

# df1.selectExpr("topic", "CAST(key AS STRING)", "CAST(value AS STRING)")\
#   .writeStream\
#   .format("kafka")\
#   .option("kafka.bootstrap.servers", "host1:port1,host2:port2")\
#   .option("checkpointLocation", "/to/HDFS-compatible/dir")\
#   .start()
    

In [39]:
# df1.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")\
#   .writeStream\
#   .format("kafka")\
#   .option("kafka.bootstrap.servers", "host1:port1,host2:port2")\
#   .option("checkpointLocation", "/to/HDFS-compatible/dir")\
#   .option("topic", "topic1")\
#   .start()

In [40]:
# COMMAND ----------
# run ----- nc -lk 9999 on terminal
socketDF = spark.readStream.format("socket")\
  .option("host", "localhost").option("port", 9999).load()

In [41]:
deviceModelStats = socketDF\
  .writeStream.queryName("socket").format("memory")\
  .outputMode("append")\
  .start()

In [42]:
for x in range(10):
    spark.sql("SELECT * FROM socket").show(5)
    sleep(5)

+-----+
|value|
+-----+
+-----+

+-----+
|value|
+-----+
+-----+

+-----+
|value|
+-----+
+-----+

+-------+
|  value|
+-------+
|     hi|
|kranthi|
+-------+

+---------------+
|          value|
+---------------+
|             hi|
|        kranthi|
|          kumar|
|   ddjfdkfjdskl|
|fereopewrewrewp|
+---------------+
only showing top 5 rows

+---------------+
|          value|
+---------------+
|             hi|
|        kranthi|
|          kumar|
|   ddjfdkfjdskl|
|fereopewrewrewp|
+---------------+
only showing top 5 rows

+---------------+
|          value|
+---------------+
|             hi|
|        kranthi|
|          kumar|
|   ddjfdkfjdskl|
|fereopewrewrewp|
+---------------+
only showing top 5 rows

+---------------+
|          value|
+---------------+
|             hi|
|        kranthi|
|          kumar|
|   ddjfdkfjdskl|
|fereopewrewrewp|
+---------------+
only showing top 5 rows

+---------------+
|          value|
+---------------+
|             hi|
|        kranthi|
| 

In [43]:
spark.streams.active

In [47]:
spark.sql("SELECT * FROM socket").show()

+-------------------+
|              value|
+-------------------+
|                 hi|
|            kranthi|
|              kumar|
|       ddjfdkfjdskl|
|    fereopewrewrewp|
| fdfdfsdjkfsdfdjfjd|
| dfdsfdjfjdlkfjdskl|
|it is working......|
+-------------------+



In [44]:
# # COMMAND ----------

# activityCounts.writeStream.trigger(processingTime='5 seconds')\
#   .format("console").outputMode("complete").start()

In [45]:
# COMMAND ----------

# activityCounts.writeStream.trigger(once=True)\
#   .format("console").outputMode("complete").start()